In [1]:
from transformers import MobileViTImageProcessor, MobileViTV2ForImageClassification
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

feature_extractor = MobileViTImageProcessor.from_pretrained("shehan97/mobilevitv2-1.0-imagenet1k-256")
model = MobileViTV2ForImageClassification.from_pretrained("shehan97/mobilevitv2-1.0-imagenet1k-256")

inputs = feature_extractor(images=image, return_tensors="pt")

outputs = model(**inputs)
logits = outputs.logits

# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

/Users/christiannikolov/miniconda3/envs/haseUNDigel/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/christiannikolov/miniconda3/envs/haseUNDigel/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/christiannikolov/miniconda3/envs/haseUNDigel/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <0B7EB158-53DC-3403-8A49-22178CAB4612> /Users/christiannikolov/miniconda3/envs/haseUNDigel/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/Users/christiannikolov/miniconda3/envs/haseUNDigel/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/christiannikolov/miniconda3/envs/haseUNDigel/lib/python3.10/site-pack

Predicted class: tabby, tabby cat


In [3]:
from PIL import Image
import requests
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [4]:
from models import load_model_and_processor_apple_model, predict_image_class_apple_model

feature_extractor, model = load_model_and_processor_apple_model()
predicted_class = predict_image_class_apple_model(image, feature_extractor, model)
print("Predicted class:", predicted_class)

Predicted class: tabby, tabby cat


In [ ]:
import os
from transformers import MobileViTImageProcessor, MobileViTV2ForImageClassification
from PIL import Image
import torch

# Load model and feature extractor
feature_extractor = MobileViTImageProcessor.from_pretrained("shehan97/mobilevitv2-1.0-imagenet1k-256")
model = MobileViTV2ForImageClassification.from_pretrained("shehan97/mobilevitv2-1.0-imagenet1k-256")

# Directory containing local images
image_folder = "haseigel-fs/data/images"

# Iterate through each image in the folder
for image_name in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_name)

    # Ensure it's a file (not a subdirectory)
    if os.path.isfile(image_path):
        try:
            # Open the image
            image = Image.open(image_path)

            # Preprocess the image
            inputs = feature_extractor(images=image, return_tensors="pt")

            # Get model predictions
            outputs = model(**inputs)
            logits = outputs.logits

            # Get the predicted class index and label
            predicted_class_idx = logits.argmax(-1).item()
            predicted_class = model.config.id2label[predicted_class_idx]

            # Print the result
            print(f"Image: {image_name} | Predicted class: {predicted_class}")
        
        except Exception as e:
            print(f"Error processing {image_name}: {e}")

Error processing 618_20200331264_202.png: Unable to infer channel dimension format
Image: 290_6522161-200x200.jpg | Predicted class: soap dispenser
Error processing 677_20200413224_155.png: Unable to infer channel dimension format
Image: 300_6332002-200x200.jpg | Predicted class: screwdriver
Image: 264_6522261-200x200.jpg | Predicted class: vacuum, vacuum cleaner
Error processing 502_FAG-image108.png: Unable to infer channel dimension format
Error processing 0_cropped-Logo_hero.webp: Unable to infer channel dimension format
Error processing 460_FAG-image108.png: Unable to infer channel dimension format
Image: 292_6522168-200x200.jpg | Predicted class: water bottle
Error processing 678_20200413113_843.png: Unable to infer channel dimension format
Image: 156_WEB-RUBI.jpg | Predicted class: swab, swob, mop
Image: 271_6524009-200x200.jpg | Predicted class: spotlight, spot
Image: 294_6411515-200x200.jpg | Predicted class: hamper
Image: 309_6332103-200x200.jpg | Predicted class: oil filter
I

In [10]:
from transformers import BeitImageProcessor, BeitForImageClassification
from PIL import Image
import torch

# Load the image
image_path = '../data/images/543_pre121389.jpg'
image = Image.open(image_path)

# Load the processor and model
processor = BeitImageProcessor.from_pretrained('microsoft/beit-base-patch16-224-pt22k-ft22k')
model = BeitForImageClassification.from_pretrained('microsoft/beit-base-patch16-224-pt22k-ft22k')

# Preprocess the image
inputs = processor(images=image, return_tensors="pt")

# Forward pass through the model
outputs = model(**inputs)
logits = outputs.logits

# Apply softmax to get probabilities
probs = torch.softmax(logits, dim=-1)

# Get the top 5 classes and their probabilities
top_5_probs, top_5_indices = torch.topk(probs, 5, dim=-1)

# Print the top 5 classes with their probabilities
for i in range(5):
    class_idx = top_5_indices[0, i].item()
    class_prob = top_5_probs[0, i].item()
    class_label = model.config.id2label[class_idx]
    print(f"Class: {class_label}, Probability: {class_prob:.4f}")

Class: pruning_knife, Probability: 0.1436
Class: folding_saw, Probability: 0.1349
Class: scow, Probability: 0.1313
Class: blade, Probability: 0.1165
Class: truncated_cone, Probability: 0.0368


In [ ]:
from urllib.request import urlopen
from PIL import Image
import timm

img = Image.open(image_path)

model = timm.create_model('resnet50.a1_in1k', pretrained=True)
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

output = model(transforms(img).unsqueeze(0))  # unsqueeze single image into batch of 1

top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=5)



In [15]:
from urllib.request import urlopen
from PIL import Image
import timm
import torch
import json

# Load ImageNet labels
with urlopen("https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json") as url:
    class_labels = json.loads(url.read().decode())

# Load and preprocess the image
image_path = '../data/images/543_pre121389.jpg'
img = Image.open(image_path)

# Load the pretrained model
model = timm.create_model('resnet50.a1_in1k', pretrained=True)
model = model.eval()

# Get model-specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

# Preprocess image and make prediction
input_tensor = transforms(img).unsqueeze(0)  # unsqueeze to make it a batch of 1
output = model(input_tensor)

# Get top 5 probabilities and class indices
top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1), k=5)

# Print top 5 classes with probabilities
for i in range(5):
    class_index = top5_class_indices[0, i].item()
    class_prob = top5_probabilities[0, i].item() * 100
    class_name = class_labels[class_index]
    print(f"Class: {class_name}, Probability: {class_prob:.2f}%")

Class: remote control, Probability: 39.24%
Class: switch, Probability: 12.26%
Class: toaster, Probability: 3.36%
Class: modem, Probability: 3.16%
Class: tape player, Probability: 2.59%


In [13]:
print(top5_class_indices)

tensor([[761, 844, 859, 662, 848]])


In [14]:
print(top5_probabilities)

tensor([[39.2404, 12.2643,  3.3607,  3.1600,  2.5863]],
       grad_fn=<TopkBackward0>)


### Test model from app

In [1]:
import sys 
sys.path.append('..')
# from app.core.image_models import MobileViTClassifier
image_path = '../data/images/temp/Wintergrillen 992x661.jpg.webp'

In [2]:
from app.core.image_models import VisionLanguageModelClassifier

model = VisionLanguageModelClassifier()
categories = ["grill", "axe", "hammer"]
prediction = model.predict(image_path, categories)

/Users/alexander/Desktop/projects/haseigel-fs/.haseigel/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/alexander/Desktop/projects/haseigel-fs/.haseigel/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [3]:
prediction

{'categories': {'grill': False, 'axe': False, 'hammer': False},
 'custom_category': 'snow'}